In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10, 8)
import collections
from io import StringIO

import numpy as np
import pandas as pd
import pydotplus  # pip install pydotplus
import seaborn as sns
from ipywidgets import Image
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
# Create dataframe with dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis=1)
    out.drop(feature_list, axis=1, inplace=True)
    return out


# Some feature values are present in train and absent in test and vice-versa.
def intersect_features(train, test):
    common_feat = list(set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ["Looks", "Alcoholic_beverage", "Eloquence", "Money_spent"]

In [4]:
df_train = {}
df_train["Looks"] = [
    "handsome",
    "handsome",
    "handsome",
    "repulsive",
    "repulsive",
    "repulsive",
    "handsome",
]
df_train["Alcoholic_beverage"] = ["yes", "yes", "no", "no", "yes", "yes", "yes"]
df_train["Eloquence"] = ["high", "low", "average", "average", "low", "high", "average"]
df_train["Money_spent"] = ["lots", "little", "lots", "little", "lots", "lots", "lots"]
df_train["Will_go"] = LabelEncoder().fit_transform(["+", "-", "+", "-", "-", "+", "+"])

df_train = create_df(df_train, features)
df_train

,Will_go,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Eloquence_low,Money_spent_little,Money_spent_lots
0,0,1,0,0,1,0,1,0,0,1
1,1,1,0,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,0,0,1
3,1,0,1,1,0,1,0,0,1,0
4,1,0,1,0,1,0,0,1,0,1
5,0,0,1,0,1,0,1,0,0,1
6,0,1,0,0,1,1,0,0,0,1


In [5]:
df_test = {}
df_test["Looks"] = ["handsome", "handsome", "repulsive"]
df_test["Alcoholic_beverage"] = ["no", "yes", "yes"]
df_test["Eloquence"] = ["average", "high", "average"]
df_test["Money_spent"] = ["lots", "little", "lots"]
df_test = create_df(df_test, features)
df_test

,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Money_spent_little,Money_spent_lots
0,1,0,1,0,1,0,0,1
1,1,0,0,1,0,1,1,0
2,0,1,0,1,1,0,0,1


In [6]:
# Some feature values are present in train and absent in test and vice-versa.
y = df_train["Will_go"]
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Alcoholic_beverage_yes,Looks_repulsive,Alcoholic_beverage_no,Eloquence_average,Money_spent_lots,Looks_handsome,Money_spent_little,Eloquence_high
0,1,0,0,0,1,1,0,1
1,1,0,0,0,0,1,1,0
2,0,0,1,1,1,1,0,0
3,0,1,1,1,0,0,1,0
4,1,1,0,0,1,0,0,0
5,1,1,0,0,1,0,0,1
6,1,0,0,1,1,1,0,0


In [18]:
tree = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state = 17)
tree.fit(df_train, y)

DecisionTreeClassifier(criterion='entropy', random_state=17)

In [25]:
feature_names = df_train.columns

def tree_graph_to_png(tree, feature_names, png_file_to_save):
    tree_str = export_graphviz(
        tree, feature_names=feature_names, filled=True, out_file=None
    )
    graph = pydotplus.graph_from_dot_data(tree_str)
    graph.write_png(png_file_to_save)
    tree_str

In [26]:
tree_graph_to_png(
    tree=tree,
    feature_names=feature_names,
    png_file_to_save="demo_decision_tree1.png",
)

In [37]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

In [61]:
# two groups
balls_left = [1 for i in range(8)] + [0 for i in range(5)]  # 8 blue and 5 yellow
balls_right = [1 for i in range(1)] + [0 for i in range(6)]  # 1 blue and 6 yellow
balls_right

[1, 0, 0, 0, 0, 0, 0]

In [95]:
def entropy(a_list):
    # entr = 0
    # leng = len(a_list)
    # set_el = set(a_list)
    # if leng in [0, 1]:
    #     return 0
    # for i in set_el:
    #     occ = a_list.count(i)
    #     entr -= occ / leng * log (occ / leng, 2)
    # return entr
    
    lst = list(a_list)
    size = len(lst)
    entropy = 0
    set_elements = len(set(lst))
    if set_elements in [0, 1]:
        return 0
    for i in set(lst):
        occ = lst.count(i)
        entropy -= occ / size * log(occ / size, 2)
    return entropy

In [96]:
print("9 blue и 11 yellow entropy is %5.2f" % entropy(balls))  # 9 blue и 11 yellow
print("8 blue и 5 yellow entropy is %5.2f" % entropy(balls_left))  # 8 blue и 5 yellow
print("1 blue и 6 yellow yellow entropy is %5.2f" % entropy(balls_right))  # 1 blue и 6 yellow
print("Entropy of a fair 6-sided die is %5.2f" % entropy([1, 2, 3, 4, 5, 6]))  # entropy of a fair 6-sided die
print("Entropy of a not fair 7-sided die is %5.2f" % entropy([1, 2, 3, 4, 5, 6, 6]))  # entropy of a not fair 7-sided die
print("Entropy of a not fair 6-sided die is %5.2f" % entropy([1, 1, 2, 2, 3, 3]))  # entropy of a not fair 6-sided die


9 blue и 11 yellow entropy is  0.99
8 blue и 5 yellow entropy is  0.96
1 blue и 6 yellow yellow entropy is  0.59
Entropy of a fair 6-sided die is  2.58
Entropy of a not fair 7-sided die is  2.52
Entropy of a not fair 6-sided die is  1.58


In [97]:
# information gain calculation
def information_gain(root, left, right):
    """ root - initial data, left and right - two partitions of initial data"""

    return (
        entropy(root)
        - 1.0 * len(left) / len(root) * entropy(left)
        - 1.0 * len(right) / len(root) * entropy(right)
    )

In [98]:
print(information_gain(balls, balls_left, balls_right))

0.16088518841412436


In [99]:
#Helper function for building a tree and evaluate information gain for optimal feature choice
def information_gains(X, y):
    """Outputs information gain when splitting with each feature"""
    out = []
    for i in X.columns:
        out.append(information_gain(y, y[X[i] == 0], y[X[i] == 1]))
    return out

In [100]:
information_gains(df_train, y)

[0.005977711423774124,
 0.12808527889139454,
 0.005977711423774124,
 0.02024420715375619,
 0.46956521111470706,
 0.12808527889139454,
 0.46956521111470706,
 0.2916919971380598]

In [106]:
#Implementation of decision tree building algorithm
def btree(X, y, features_names):
    # 1 Finding best feature
    
    clf = information_gains(X, y)
    best_feat_id = clf.index(max(clf))
    best_feature = feature_names[best_feat_id]
    print(f"Best feature to split: {best_feature}")
    
    # 2 Split set for left and right branches
    
    x_left = X[X.iloc[:, best_feat_id] == 0]
    x_right = X[X.iloc[:, best_feat_id] == 1]
    print(f"Samples: {len(x_left)} (left) and {len(x_right)} (right)")
    
    
    y_left = y[X.iloc[:, best_feat_id] == 0]
    y_right = y[X.iloc[:, best_feat_id] == 1]
    entropy_left = entropy(y_left)
    entropy_right = entropy(y_right)
    print(f"Entropy: {entropy_left} (left) and {entropy_right} (right)")
    print("_" * 30 + "\n")
    
    # 3 Recursive split until entropy become zero
    
    if entropy_left != 0:
        print(f"Splitting the left group with {len(x_left)} samples:")
        btree(x_left, y_left, feature_names)
    if entropy_right != 0:
        print(f"Splitting the right group with {len(x_right)} samples:")
        btree(x_right, y_right, feature_names)
        

In [107]:
    btree(df_train, y, df_train.columns)

Best feature to split: Money_spent_lots
Samples: 2 (left) and 5 (right)
Entropy: 0 (left) and 0.7219280948873623 (right)
______________________________

Splitting the right group with 5 samples:
Best feature to split: Looks_repulsive
Samples: 3 (left) and 2 (right)
Entropy: 0 (left) and 1.0 (right)
______________________________

Splitting the right group with 2 samples:
Best feature to split: Eloquence_high
Samples: 1 (left) and 1 (right)
Entropy: 0 (left) and 0 (right)
______________________________

